# Actividad: Regresión Líneal CDMX

## Carga de librerías y dataframe

In [543]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [544]:
# Cargar archivo CSV usando pandas
df = pd.read_csv('CDMX_Limpio.csv')
df.head(2)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\2743940659.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('CDMX_Limpio.csv')


,Unnamed: 0.1,last_scraped,name,description,neighborhood_overview,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d
0,0,2024-09-25,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Mall, Interloma Park and the Lion Des...","Mexico City, Mexico",a few days or more,0%,Without information,f,...,7.0,1.0,7.0,28,58,88,363,0.0,0.0,0.0
1,1,2024-09-26,Condesa Haus,A new concept of hosting in mexico through a b...,Nothing to say,"Mexico City, Mexico",within an hour,100%,93%,t,...,365.0,1.0,365.0,1,24,54,325,64.0,0.0,0.0


## Creación de los dataframes con las variables a utilizar

In [545]:
# Convertimos las variables price y review_scores_rating a numérico
df['price'] = pd.to_numeric(df['price'].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')
df['review_scores_rating'] = pd.to_numeric(df['review_scores_rating'].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')

In [546]:
# Rellenamos los valores nulos de review_scores_rating con bffill y ffill
df['review_scores_rating'] = df['review_scores_rating'].fillna(method='ffill')
df['review_scores_rating'] = df['review_scores_rating'].fillna(method='bfill')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3690231316.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['review_scores_rating'] = df['review_scores_rating'].fillna(method='ffill')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3690231316.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['review_scores_rating'] = df['review_scores_rating'].fillna(method='bfill')


In [547]:
# Rellenamos los valores NaN con bfill
df['price'].fillna(method='bfill', inplace=True)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3229511809.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(method='bfill', inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3229511809.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['price'].fillna(method='bfill', inplace=True)


In [548]:
# Cambiamos los valores '?' el valor que mas se repite en la columna (t o f) para la columna host_is_superhost
df['host_is_superhost'] = df['host_is_superhost'].replace({'?': df['host_is_superhost'].mode()[0]})
df['has_availability'] = df['has_availability'].replace({'?': df['has_availability'].mode()[0]})

In [549]:
# Cambiamos has no information por f en la columna has_availability
df['has_availability'] = df['has_availability'].replace({'Has no information': 'f'})

In [550]:
# Creamos una copia del DataFrame para las variables dependientes: host_is_superhost, host_has_profile_pic, host_identity_verified, has_availability, instant_bookable, price, property_type, accommodates, room_type, review_scores_rating.
df_Dep = df[['host_is_superhost', 'host_has_profile_pic', 
             'host_identity_verified', 'has_availability', 
             'instant_bookable', 'price', 'property_type', 
             'accommodates', 'room_type', 'review_scores_rating']].copy()
df_Dep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26582 entries, 0 to 26581
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   host_is_superhost       26582 non-null  object 
 1   host_has_profile_pic    26582 non-null  object 
 2   host_identity_verified  26582 non-null  object 
 3   has_availability        26582 non-null  object 
 4   instant_bookable        26582 non-null  object 
 5   price                   26582 non-null  float64
 6   property_type           26582 non-null  object 
 7   accommodates            26582 non-null  float64
 8   room_type               26582 non-null  object 
 9   review_scores_rating    26582 non-null  float64
dtypes: float64(3), object(7)
memory usage: 2.0+ MB


In [551]:
# Convertimos instant_bookable a binario
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})

In [552]:
# Convertimos las variables host_response_rate, review_scores_communication, review_scores_cleanliness, review_scores_value a numéricas
df['host_response_rate'] = pd.to_numeric(df['host_response_rate'].str.replace('%', ''), errors='coerce')

In [553]:
# Convertimos la columna review_scores_communication a numérica reemplazando Does not say por NaN
df['review_scores_communication'] = df['review_scores_communication'].replace('Does not say', np.nan)
# Rellenamos los valores NaN con bfill y ffill
df['review_scores_communication'].fillna(method='ffill', inplace=True)
df['review_scores_communication'].fillna(method='bfill', inplace=True)
df['host_response_rate'] = pd.to_numeric(df['host_response_rate'])


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3193969631.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['review_scores_communication'].fillna(method='ffill', inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3193969631.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['review_scores_communication'].fillna(method='ffill', inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3193969631.py:5: FutureWa

In [554]:
df['review_scores_communication'].head(2)

0    4.78
1    4.78
Name: review_scores_communication, dtype: object

In [555]:
# Revisamos el numero de valores nulos en las columnas host_response_rate, review_scores_communication, review_scores_cleanliness, review_scores_value
df[['host_response_rate', 'review_scores_communication', 'review_scores_cleanliness', 'review_scores_value']].isnull().sum()

host_response_rate             0
review_scores_communication    0
review_scores_cleanliness      0
review_scores_value            0
dtype: int64

In [556]:
# Convertimos en numéricas las columnas host_response_rate, review_scores_communication, review_scores_cleanliness, review_scores_value
df['review_scores_cleanliness'] = pd.to_numeric(df['review_scores_cleanliness'].astype(str).str.replace('%', ''), errors='coerce')
df['review_scores_value'] = pd.to_numeric(df['review_scores_value'].astype(str).str.replace('%', ''), errors='coerce')
df['review_scores_communication'] = pd.to_numeric(df['review_scores_communication'].astype(str).str.replace('%', ''), errors='coerce')

In [557]:
# Convertimos a float las columnas host_response_rate, review_scores_communication, review_scores_cleanliness, review_scores_value
df['host_response_rate'] = df['host_response_rate'].astype(float)
df['review_scores_communication'] = df['review_scores_communication'].astype(float)
df['review_scores_cleanliness'] = df['review_scores_cleanliness'].astype(float)

In [558]:
# Rellenamos los valores nulos de review_scores_cleanliness y review_scores_value con bffill y ffill
df['review_scores_cleanliness'] = df['review_scores_cleanliness'].fillna(method='ffill')
df['review_scores_cleanliness'] = df['review_scores_cleanliness'].fillna(method='bfill')
df['review_scores_value'] = df['review_scores_value'].fillna(method='ffill')
df['review_scores_value'] = df['review_scores_value'].fillna(method='bfill')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3453679345.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['review_scores_cleanliness'] = df['review_scores_cleanliness'].fillna(method='ffill')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3453679345.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['review_scores_cleanliness'] = df['review_scores_cleanliness'].fillna(method='bfill')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3453679345.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['review_scores_value'] = df['review_scores_value'].fillna(method='ffill')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9624\3453679345.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise 

In [559]:
# Creamos una copia del DataFrame para las variables independientes: host_response_rate, number_of_reviews, review_scores_rating, review_scores_communication, availability_365, minimum_nights, maximum_nights, availability_30, accommodates, bedrooms, bathrooms, price, review_scores_cleanliness, review_scores_value, instant_bookable.
df_Ind = df[['host_response_rate', 'number_of_reviews',
             'review_scores_rating', 'review_scores_communication', 
             'availability_365', 'minimum_nights', 'maximum_nights', 
             'availability_30', 'accommodates', 'bedrooms', 
             'bathrooms', 'beds', 'price', 'review_scores_cleanliness', 
             'review_scores_value', 'instant_bookable']].copy()
df_Ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26582 entries, 0 to 26581
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_response_rate           26582 non-null  float64
 1   number_of_reviews            26582 non-null  float64
 2   review_scores_rating         26582 non-null  float64
 3   review_scores_communication  26582 non-null  float64
 4   availability_365             26582 non-null  int64  
 5   minimum_nights               26582 non-null  float64
 6   maximum_nights               26582 non-null  int64  
 7   availability_30              26582 non-null  int64  
 8   accommodates                 26582 non-null  float64
 9   bedrooms                     26582 non-null  float64
 10  bathrooms                    26582 non-null  float64
 11  beds                         26582 non-null  float64
 12  price                        26582 non-null  float64
 13  review_scores_cl

## Convertir a dicótomicas algunas variables dependientes

In [560]:
# Convertimos la variable price a dicotómica a partir de su promedio 
df_Dep['price'] = df_Dep['price'].apply(lambda x: 1 if x > df_Dep['price'].mean() else 0)
df_Dep['price'] = df_Dep['price'].replace({1: 'higher', 0: 'lower'})
df_Dep['price'].value_counts()

price
lower     19680
higher     6902
Name: count, dtype: int64

In [561]:
# Convertimos la variable review_scores_rating a dicotómica a partir de su promedio
df_Dep['review_scores_rating'] = df_Dep['review_scores_rating'].apply(lambda x: 1 if x > df_Dep['review_scores_rating'].mean() else 0)
df_Dep['review_scores_rating'] = df_Dep['review_scores_rating'].replace({1: 'higher', 0: 'lower'}) 
df_Dep['review_scores_rating'].value_counts()

review_scores_rating
higher    17815
lower      8767
Name: count, dtype: int64

In [562]:
# Convertimos la variable accommodates a dicotómica a partir de su promedio
df_Dep['accommodates'] = df_Dep['accommodates'].apply(lambda x: 1 if x > df_Dep['accommodates'].mean() else 0)
df_Dep['accommodates'] = df_Dep['accommodates'].replace({1: 'higher', 0: 'lower'})
df_Dep['accommodates'].value_counts()

accommodates
lower     15005
higher    11577
Name: count, dtype: int64

In [563]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['property_type'])
df_Dep['property_type'].value_counts()

property_type
Entire rental unit             9971
Entire condo                   3423
Private room in rental unit    3034
Private room in home           2266
Entire loft                    1515
                               ... 
Private room in floor             1
Earthen home                      1
Entire in-law                     1
Shared room in dorm               1
Tower                             1
Name: count, Length: 92, dtype: int64

In [564]:
# Convertimos la variable property_type a dicotómica con Entire rental unit y el resto other
df_Dep['property_type'] = df_Dep['property_type'].apply(lambda x: 1 if x == 'Entire rental unit' else 0)
df_Dep['property_type'] = df_Dep['property_type'].replace({1: 'entire', 0: 'other'})
df_Dep['property_type'].value_counts()

property_type
other     16611
entire     9971
Name: count, dtype: int64

In [565]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
df_Dep['room_type'].value_counts()

room_type
Entire home/apt    17363
Private room        8693
Shared room          401
Hotel room           125
Name: count, dtype: int64

In [566]:
# Convertimos la variable room_type a dicotómica con Entire rental unit y el resto other
df_Dep['room_type'] = df_Dep['room_type'].apply(lambda x: 1 if x == 'Entire home/apt' else 0)
df_Dep['room_type'] = df_Dep['room_type'].replace({1: 'apt', 0: 'other'})
df_Dep['room_type'].value_counts()

room_type
apt      17363
other     9219
Name: count, dtype: int64

## Regresión líneal de las variables

### host_is_superhost

In [567]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['host_is_superhost']

In [568]:
# Revisamos el numero de nulos en las variables independientes
Vars_Indep.isnull().sum()

host_response_rate      0
number_of_reviews       0
review_scores_rating    0
dtype: int64

In [569]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [570]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [571]:
#se escalan todos los datos
escalar = StandardScaler()

In [572]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [573]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [574]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [575]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 't', 't'], dtype=object)

In [576]:
unico = np.unique(df_Dep['host_is_superhost'])
unico

array(['f', 't'], dtype=object)

In [577]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4246  621]
 [1892 1216]]


In [578]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.6619488296135003


In [579]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.6848902821316615


In [580]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.8724059995890693


### host_has_profile_pic

In [581]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_communication']]
Vars_Dep = df_Dep['host_has_profile_pic']

In [582]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [583]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [584]:
#se escalan todos los datos
escalar = StandardScaler()

In [585]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [586]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [587]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [588]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [589]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  136]
 [   0 7839]]


In [590]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9829467084639498


In [591]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9829467084639498


In [592]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### host_identity_verified

In [593]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'availability_365']]
Vars_Dep = df_Dep['host_identity_verified']

In [594]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [595]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [596]:
#se escalan todos los datos
escalar = StandardScaler()

In [597]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [598]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [599]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [600]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [601]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  353]
 [   0 7622]]


In [602]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9557366771159874


In [603]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9557366771159874


In [604]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### has_availability

In [605]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['minimum_nights', 'maximum_nights', 'availability_30', 'availability_365']]
Vars_Dep = df_Dep['has_availability']

In [606]:
df_Dep['has_availability'].value_counts()

has_availability
t    25533
f     1049
Name: count, dtype: int64

In [607]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [608]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [609]:
#se escalan todos los datos
escalar = StandardScaler()

In [610]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [611]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [612]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [613]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [614]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  304]
 [   0 7671]]


In [615]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9618808777429467


In [616]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9618808777429467


In [617]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### instant_bookable

In [618]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['instant_bookable']

In [619]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [620]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [621]:
#se escalan todos los datos
escalar = StandardScaler()

In [622]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [623]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [624]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [625]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [626]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4648  153]
 [2955  219]]


In [627]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.5887096774193549


In [628]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.610282131661442


In [629]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9681316392418247


### price

In [630]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['price']

In [631]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [632]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [633]:
#se escalan todos los datos
escalar = StandardScaler()

In [634]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [635]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [636]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [637]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['lower', 'lower', 'lower', ..., 'higher', 'lower', 'lower'],
      dtype=object)

In [638]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 721 1297]
 [ 424 5533]]


In [639]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.6296943231441048


In [640]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.7842006269592476


In [641]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9288232331710593


### property_type

In [642]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'price', 'review_scores_rating']]
Vars_Dep = df_Dep['property_type']

In [643]:
# Verificamos que las variables independientes no tengan valores nulos
print(Vars_Indep.isnull().sum())

accommodates            0
bedrooms                0
price                   0
review_scores_rating    0
dtype: int64


In [644]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [645]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [646]:
#se escalan todos los datos
escalar = StandardScaler()

In [647]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [648]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [649]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [650]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['other', 'other', 'other', ..., 'other', 'other', 'other'],
      dtype=object)

In [651]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 715 2315]
 [ 545 4400]]


In [652]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="entire")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.5674603174603174


In [653]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.6413793103448275


In [654]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="other")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.8897876643073812


### accommodates

In [655]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['bedrooms', 'bathrooms', 'beds', 'price', 'number_of_reviews']]
Vars_Dep = df_Dep['accommodates']

In [656]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [657]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [658]:
#se escalan todos los datos
escalar = StandardScaler()

In [659]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [660]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [661]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [662]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['lower', 'lower', 'lower', ..., 'higher', 'lower', 'lower'],
      dtype=object)

In [663]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[2743  750]
 [ 744 3738]]


In [664]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.7866360768568971


In [665]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.8126645768025078


In [666]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.8340026773761714


### room_type

In [667]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'price', 'instant_bookable']]
Vars_Dep = df_Dep['room_type']

In [668]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [669]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [670]:
#se escalan todos los datos
escalar = StandardScaler()

In [671]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [672]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [673]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [674]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['apt', 'apt', 'apt', ..., 'other', 'other', 'apt'], dtype=object)

In [675]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4032 1184]
 [ 647 2112]]


In [676]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="apt")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.861722590297072


In [677]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.7704075235109717


In [678]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="other")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.765494744472635


### review_scores_rating

In [679]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['number_of_reviews', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_value']]
Vars_Dep = df_Dep['review_scores_rating']

In [680]:
# Revisamos el numero de nulos en las variables independientes
Vars_Indep.isnull().sum()

number_of_reviews              0
review_scores_cleanliness      0
review_scores_communication    0
review_scores_value            0
dtype: int64

In [681]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [682]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [683]:
#se escalan todos los datos
escalar = StandardScaler()

In [684]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [685]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [686]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [687]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['higher', 'lower', 'lower', ..., 'higher', 'lower', 'higher'],
      dtype=object)

In [688]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[5001  329]
 [ 723 1922]]


In [689]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.8736897274633124


In [690]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.8680877742946709


In [691]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.7266540642722117
